In [1]:
library(tidyverse)
library(plotROC)

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.1.1       ✔ purrr   0.3.2  
✔ tibble  2.1.1       ✔ dplyr   0.8.0.1
✔ tidyr   0.8.3       ✔ stringr 1.4.0  
✔ readr   1.3.1       ✔ forcats 0.4.0  
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [2]:
`%+replace%` <- ggplot2::`%+replace%`

theme_zietzm <- function(base_size = 11.5, base_family = "") {
    # Starts with theme_bw and then modify some parts
    # Theme options are documentated at http://docs.ggplot2.org/current/theme.html
    ggplot2::theme_classic(base_size = base_size, base_family = base_family) %+replace%
    ggplot2::theme(

        strip.background  = ggplot2::element_rect(fill = NA, colour = 'grey90', size = 0),
        strip.text = element_text(vjust = 1, size = 10),
        plot.margin       = ggplot2::margin(t=2, r=2, b=2, l=2, unit='pt'),
        legend.spacing     = grid::unit(0.1, 'cm'),
        legend.key        = ggplot2::element_blank(),
        panel.border=element_rect(fill = NA, color = 'black', size = 0.5), 
        axis.line=element_line(size=0),
    )
}

In [3]:
plot_df <- read_tsv('plot_info.tsv')

Parsed with column specification:
cols(
  edge_test = col_double(),
  feature = col_character(),
  value = col_double(),
  feature_information = col_character(),
  feature_group = col_character(),
  data_source = col_character(),
  task = col_character()
)


In [4]:
full <- (
    ggplot(
        plot_df %>% filter(!grepl('p_', feature)),
        aes(d = edge_test, m = value, color = feature_group, linetype = feature_information)
    )
    + geom_roc(n.cuts = 0)
    + geom_abline(slope = 1, intercept = 0, linetype = 'dashed', color = 'grey')
    
    + geom_roc(data = plot_df %>% filter(grepl('p_', feature)), 
               increasing = F, n.cuts = 0)
    + scale_linetype_manual(values=c('corrected feature'='dotdash', 'feature'='solid', 'no edge info'='dashed'))
    + facet_grid(cols=vars(data_source), rows=vars(task))
    + xlab("False positive rate")
    + ylab("True positive rate")
    + theme_zietzm()
)
rm(plot_df)

In [5]:
ggsave(file = '../../img/4_ROC/reconstruction_systematic.png', full, width = 9, height = 6, dpi = 500)